In [2]:
import coralme
import pandas
from coralme.solver.solver import ME_NLP
import cobra
import numpy
from coralme.core.optimization import _check_options
from coralme.core.optimization import construct_lp_problem

#from coralme.core.optimization import fva
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'
pandas.set_option('display.max_colwidth', None)
import inspect

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.metrics import r2_score
import math

In [3]:
coralme.__version__

'1.2.1'

In [2]:
def fva(model,
	reaction_list, fraction_of_optimum, mu_fixed = None, objective = 'biomass_dilution',
	max_mu = 2.8100561374051836, min_mu = 0., maxIter = 100, lambdify = True,
	tolerance = 1e-6, precision = 'quad', verbose = True):

	"""
	Determine the minimum and maximum flux value for each reaction constrained
	to a fraction of the current growth rate (default = 1.0)

	Parameters
	----------
	reaction_list : list of cobra.Reaction or str, optional
		List of reactions IDs and/or reaction objects
	fraction_of_optimum : float, optional
		Must be <= 1.0. Requires that the objective value is at least the
		fraction times maximum objective value. A value of 0.85 for instance
		means that the objective has to be at least at 85% percent of its
		maximum (default 1.0).
	mu_fixed : float, optional
		Set it to avoid the optimization of a ME-model. The growth rate must
		be feasible. If not, the ME-model will be optimized with the following
		options:

		max_mu : float, optional
			Maximum growth rate for initializing the growth rate binary search (GRBS).
		min_mu : float, optional
			Minimum growth rate for initializing GRBS.
		maxIter : int
			Maximum number of iterations for GRBS.
		lambdify : bool
			If True, returns a dictionary of lambda functions for each symbolic
			stoichiometric coefficient
		tolerance : float
			Tolerance for the convergence of GRBS.
		precision : str, {"quad", "double", "dq", "dqq"}
			Precision (quad or double precision) for the GRBS

	verbose : bool
		If True, allow printing.
	"""

	# max_mu is constrained by the fastest-growing bacterium (14.8 doubling time)
	# https://www.nature.com/articles/s41564-019-0423-8

	# check options
	keys={}
	keys, tolerance, precision = _check_options(model = model, keys = keys, tolerance = tolerance, precision = precision)
	fraction_of_optimum = fraction_of_optimum if fraction_of_optimum <= 1.0 and fraction_of_optimum >= 0.0 else 1.0
	if isinstance(reaction_list, str):
		reaction_list = [reaction_list]

	# populate with stoichiometry, no replacement of mu's
	if hasattr(model, 'construct_lp_problem'):
		# check if the ME-model has a solution
		if mu_fixed is not None and not hasattr(model, 'solution'):
			model.optimize(max_mu = max_mu, min_mu = min_mu, maxIter = maxIter, lambdify = lambdify,
				tolerance = tolerance, precision = precision, verbose = verbose)

		# set mu_fixed for replacement in a ME-model.
		mu_fixed = model.solution.fluxes.get(objective, mu_fixed) * fraction_of_optimum

		# get mathematical representation
		Sf, Se, lb, ub, b, c, cs, atoms, lambdas, Lr, Lm = construct_lp_problem(model, lambdify = lambdify)
	else:
		# not a ME-model, and objective bounds usually are (0, 1000)
		if model.reactions.has_id(objective):
			model.reactions.get_by_id(objective).lower_bound = mu_fixed * fraction_of_optimum
			model.reactions.get_by_id(objective).upper_bound = mu_fixed
		else:
			raise ValueError('Objective reaction \'{:s}\' not in the M-model.'.format(objective))

		# get mathematical representation
		Sf, Se, lb, ub, b, c, cs, atoms, lambdas, Lr, Lm = construct_lp_problem(model)

	if verbose:
		print('Running FVA for {:d} reactions. Maximum growth rate fixed to {:g}'.format(len(reaction_list), mu_fixed))

	me_nlp = coralme.solver.solver.ME_NLP(Sf, Se, b, c, lb, ub, cs, atoms, lambdas)

	# We need only reaction objects
	rxns_fva = []
	for rxn in reaction_list:
		if isinstance(rxn, str) and model.reactions.has_id(rxn):
			rxns_fva.append(model.reactions.get_by_id(rxn))
		else:
			rxns_fva.append(rxn)

	obj_inds0 = [ model.reactions.index(rxn) for rxn in rxns_fva for j in range(0, 2) ]
	obj_coeffs = [ ci for rxn in rxns_fva for ci in (1.0, -1.0) ]

	# varyME is a specialized method for multiple min/maximization problems
	obj_inds0, nVary, obj_vals = me_nlp.varyme(mu_fixed, obj_inds0, obj_coeffs, basis = None, verbosity = verbose)

	# Return result consistent with cobrapy FVA
	fva_result = {
		(model.reactions[obj_inds0[2*i]].id): {
			'maximum':obj_vals[2*i],
			'minimum':obj_vals[2*i+1]
			} for i in range(0, nVary//2) }

	return pandas.DataFrame(fva_result).T

In [3]:
m_model=cobra.io.read_sbml_model("./inputs/m_model.xml")

Set parameter Username
Set parameter LicenseID to value 2654766
Academic license - for non-commercial use only - expires 2026-04-20


In [4]:
m_model.reactions.get_by_id('EX_nh4_e').lower_bound=-5
m_model.reactions.get_by_id('EX_succ_e').lower_bound=-5
assert m_model.optimize().objective_value > 1e-3


In [5]:
model_ = coralme.core.model.MEModel.from_cobra(m_model)

Read LP format model from file /tmp/tmpn9pb5o3m.lp
Reading time = 0.01 seconds
: 1786 rows, 4314 columns, 17098 nonzeros


In [6]:
model_.optimize()
sol=model_.solution

The MINOS and quad MINOS solvers are a courtesy of Prof Michael A. Saunders. Please cite Ma, D., Yang, L., Fleming, R. et al. Reliable and efficient solution of genome-scale models of Metabolism and macromolecular Expression. Sci Rep 7, 40863 (2017). https://doi.org/10.1038/srep40863



In [7]:
sol

,fluxes,reduced_costs
CYTDK2,0.000000e+00,-6.464732e-35
XPPT,0.000000e+00,-5.641817e-03
HXPRT,0.000000e+00,-5.641817e-03
NDPK5,3.481538e-54,3.609692e-35
SHK3Dr,2.884806e-01,-5.549088e-35
...,...,...
NO3R1bpp,3.014747e+00,0.000000e+00
BIOMASS_WT,0.000000e+00,-8.360825e-01
CLPNH161pp,0.000000e+00,-4.936590e-03
CLPNS161pp,1.122849e-04,-2.429422e-34


In [8]:
# Calculating non FVA fluxes
m_model_fva = fva(model_, model_.reactions,fraction_of_optimum=1.0, objective = "BIOMASS_core")

Running FVA for 2157 reactions. Maximum growth rate fixed to 0.209847


In [9]:
m_model_fva.to_csv("m_model.csv")